In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import os
from tqdm import tqdm
import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.cluster import KMeans, DBSCAN
from scipy.spatial import distance
import networkx as nxp
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [5]:
# Cell 2: Implémentation du modèle et de l'entraînement
import os
import cv2
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# Vérifier la disponibilité du GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU disponible. Utilisation du GPU pour l\'entraînement.')
    # Afficher les informations du GPU
    print(f"GPU utilisé: {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU totale: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Mémoire GPU disponible: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB utilisée")

    # Optimisations pour performance GPU
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
else:
    device = torch.device('cpu')
    print('GPU non disponible. Utilisation du CPU pour l\'entraînement.')

# Nouveaux chemins pour les données
images_dir = '/content/drive/MyDrive/data/montagne/train/images'
labels_dir = '/content/drive/MyDrive/data/montagne/train/labels'

# Vérifier que les répertoires existent
print(f"Images directory exists: {os.path.exists(images_dir)}")
print(f"Labels directory exists: {os.path.exists(labels_dir)}")

# Créer une classe de dataset personnalisée pour le format YOLO
class YOLODataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        if os.path.exists(images_dir):
            self.image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
            print(f"Found {len(self.image_files)} images")
        else:
            print(f"Warning: Images directory {images_dir} not found")
            self.image_files = []

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_file = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_file)

        image = cv2.imread(img_path)
        if image is None:
            print(f"Warning: Could not read image {img_path}")
            # Retourner une image noire en cas d'échec de lecture
            image = np.zeros((640, 640, 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label_file = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(self.labels_dir, label_file)

        boxes = []
        classes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    try:
                        data = line.strip().split()
                        if len(data) >= 5:  # S'assurer qu'il y a assez de données
                            class_id = int(data[0])
                            x_center, y_center, width, height = map(float, data[1:5])
                            # Vérification de validité des coordonnées
                            if 0 <= x_center <= 1 and 0 <= y_center <= 1 and width > 0 and height > 0:
                                classes.append(class_id)
                                boxes.append([x_center, y_center, width, height])
                    except (ValueError, IndexError) as e:
                        print(f"Warning: Error parsing line in {label_path}: {e}")
                        continue

        boxes = np.array(boxes, dtype=np.float32)
        classes = np.array(classes, dtype=np.int64)

        if self.transform:
            try:
                if len(boxes) > 0:
                    h, w, _ = image.shape
                    transformed_boxes = []
                    for box in boxes:
                        x_center, y_center, width, height = box
                        x1 = (x_center - width/2) * w
                        y1 = (y_center - height/2) * h
                        x2 = (x_center + width/2) * w
                        y2 = (y_center + height/2) * h
                        transformed_boxes.append([x1, y1, x2, y2])

                    transformed = self.transform(image=image, bboxes=transformed_boxes, class_labels=classes)
                    image = transformed['image']

                    if transformed['bboxes']:
                        h, w = transformed['image'].shape[1:3] if isinstance(transformed['image'], torch.Tensor) else transformed['image'].shape[:2]
                        boxes = []
                        for box in transformed['bboxes']:
                            x1, y1, x2, y2 = box
                            x_center = ((x1 + x2) / 2) / w
                            y_center = ((y1 + y2) / 2) / h
                            width = (x2 - x1) / w
                            height = (y2 - y1) / h
                            boxes.append([x_center, y_center, width, height])
                        boxes = np.array(boxes, dtype=np.float32)
                        classes = np.array(transformed['class_labels'], dtype=np.int64)
                    else:
                        boxes = np.zeros((0, 4), dtype=np.float32)
                        classes = np.array([], dtype=np.int64)
                else:
                    transformed = self.transform(image=image)
                    image = transformed['image']
            except Exception as e:
                print(f"Error during transformation: {e}")
                # En cas d'erreur, retourner une image transformée sans boîtes
                transformed = self.transform(image=image)
                image = transformed['image']
                boxes = np.zeros((0, 4), dtype=np.float32)
                classes = np.array([], dtype=np.int64)

        return {
            'image': image,
            'boxes': boxes,
            'classes': classes
        }

# Définir les transformations pour l'entraînement
train_transforms = A.Compose([
    A.RandomResizedCrop(size=(640, 640)),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

val_transforms = A.Compose([
    A.Resize(height=640, width=640),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

# Créer les datasets
train_dataset = YOLODataset(images_dir, labels_dir, transform=train_transforms)

if len(train_dataset) == 0:
    print("Dataset is empty! Please check your data paths.")
else:
    def collate_fn(batch):
        return {
            'image': torch.stack([item['image'] for item in batch]),
            'boxes': [item['boxes'] for item in batch],
            'classes': [item['classes'] for item in batch]
        }

    # Optimisations pour le DataLoader avec GPU
    num_workers = 2 if device.type == 'cuda' else 0
    pin_memory = True if device.type == 'cuda' else False

    train_loader = DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory,
        collate_fn=collate_fn
    )

    # Analyser les classes dans le dataset
    all_classes = []
    for i in range(len(train_dataset)):
        try:
            sample = train_dataset[i]
            if 'classes' in sample and len(sample['classes']) > 0:
                all_classes.extend(sample['classes'].tolist())
        except Exception as e:
            print(f"Warning: Error processing sample {i}: {e}")
            continue

    unique_classes = set(all_classes)
    num_classes = len(unique_classes) + 1  # +1 pour la classe de fond
    print(f"Detected {num_classes-1} unique classes: {unique_classes}")

    # Définir le modèle
    def get_model(num_classes):
        model = models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model

    model = get_model(num_classes)
    model.to(device)

    # Définir l'optimiseur et le scheduler
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    # Nombre d'époques augmenté à 100
    num_epochs = 100

    # Sauvegarde du meilleur modèle
    best_loss = float('inf')

    # Boucle d'entraînement
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        epoch_count = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            try:
                images = list(img.to(device) for img in batch['image'])
                targets = []

                for i in range(len(images)):
                    target = {}
                    boxes = batch['boxes'][i]

                    # Ignorer les échantillons sans boxes valides dès le début
                    if len(boxes) == 0:
                        target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                        target['labels'] = torch.tensor([], dtype=torch.int64).to(device)
                        targets.append(target)
                        continue

                    # Conversion et validation des boxes
                    boxes = torch.tensor(boxes, dtype=torch.float32)

                    # Vérifier la taille correcte des boxes
                    if boxes.shape[0] == 0 or boxes.dim() != 2 or boxes.shape[1] != 4:
                        target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                        target['labels'] = torch.tensor([], dtype=torch.int64).to(device)
                        targets.append(target)
                        continue

                    try:
                        x_centers, y_centers, widths, heights = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]

                        # Vérifier la validité des dimensions
                        valid_boxes = (widths > 0) & (heights > 0) & (x_centers > 0) & (x_centers < 1) & (y_centers > 0) & (y_centers < 1)

                        if not torch.any(valid_boxes):
                            target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                            target['labels'] = torch.tensor([], dtype=torch.int64).to(device)
                            targets.append(target)
                            continue

                        # Filtrer les boxes valides
                        x_centers = x_centers[valid_boxes]
                        y_centers = y_centers[valid_boxes]
                        widths = widths[valid_boxes]
                        heights = heights[valid_boxes]

                        # Conversion au format (x1, y1, x2, y2)
                        x1s = (x_centers - widths/2)
                        y1s = (y_centers - heights/2)
                        x2s = (x_centers + widths/2)
                        y2s = (y_centers + heights/2)

                        # Vérifier que les dimensions de l'image sont disponibles
                        if images[i].dim() > 2:
                            h, w = images[i].shape[1:3]

                            # Mise à l'échelle des coordonnées
                            x1s = x1s * w
                            y1s = y1s * h
                            x2s = x2s * w
                            y2s = y2s * h

                            # Créer les tenseurs de boxes et labels
                            converted_boxes = torch.tensor([[x1, y1, x2, y2] for x1, y1, x2, y2 in zip(x1s, y1s, x2s, y2s)], dtype=torch.float32).to(device)

                            # Vérifier si batch['classes'][i] a des éléments valides
                            if i < len(batch['classes']) and len(batch['classes'][i]) > 0:
                                valid_classes_idx = valid_boxes.nonzero().flatten().cpu().numpy()
                                if len(valid_classes_idx) > 0 and len(batch['classes'][i]) > max(valid_classes_idx):
                                    valid_classes = batch['classes'][i][valid_classes_idx]
                                    if len(valid_classes) > 0:
                                        target['boxes'] = converted_boxes
                                        target['labels'] = torch.tensor(valid_classes, dtype=torch.int64).to(device)
                                    else:
                                        target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                                        target['labels'] = torch.tensor([], dtype=torch.int64).to(device)
                                else:
                                    target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                                    target['labels'] = torch.tensor([], dtype=torch.int64).to(device)
                            else:
                                target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                                target['labels'] = torch.tensor([], dtype=torch.int64).to(device)
                        else:
                            # Si l'image n'a pas les dimensions attendues
                            target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                            target['labels'] = torch.tensor([], dtype=torch.int64).to(device)
                    except Exception as e:
                        print(f"Erreur lors du traitement des boxes : {e}")
                        target['boxes'] = torch.zeros((0, 4), dtype=torch.float32).to(device)
                        target['labels'] = torch.tensor([], dtype=torch.int64).to(device)

                    targets.append(target)

                # Vérifier que tous les targets ont les champs requis
                if all('boxes' in target and 'labels' in target for target in targets):
                    # Forward pass
                    loss_dict = model(images, targets)
                    losses = sum(loss for loss in loss_dict.values())

                    # Backward pass
                    optimizer.zero_grad()
                    losses.backward()
                    optimizer.step()

                    total_loss += losses.item()
                    epoch_count += 1
                else:
                    print("Skipping batch due to invalid target shapes")
            except Exception as e:
                print(f"Error processing batch: {e}")
                # Libérer la mémoire en cas d'erreur pour éviter les fuites
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                continue

        # Mise à jour du scheduler et affichage du loss
        lr_scheduler.step()
        mean_loss = total_loss / max(1, epoch_count)
        print(f"Epoch {epoch+1}/{num_epochs} Loss: {mean_loss:.4f}")

        # Sauvegarde du meilleur modèle
        if mean_loss < best_loss:
            best_loss = mean_loss
            torch.save(model.state_dict(), 'best_summit_seeker_model.pth')
            print(f"New best model saved with loss: {best_loss:.4f}")

        # Sauvegarde du modèle toutes les 10 époques
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(), f'summit_seeker_model_epoch{epoch+1}.pth')
            print(f"Checkpoint saved at epoch {epoch+1}")

        # Libérer la mémoire GPU à la fin de chaque époque
        if device.type == 'cuda':
            torch.cuda.empty_cache()

    # Sauvegarde du modèle final
    torch.save(model.state_dict(), 'summit_seeker_model_mountain.pth')
    print("Final model saved successfully!")

GPU disponible. Utilisation du GPU pour l'entraînement.
GPU utilisé: Tesla T4
Mémoire GPU totale: 15.83 GB
Mémoire GPU disponible: 0.52 GB utilisée
Images directory exists: True
Labels directory exists: True
Found 45 images
Detected 1 unique classes: {0}


Epoch 1/100: 100%|██████████| 12/12 [00:08<00:00,  1.38it/s]


Epoch 1/100 Loss: 1.0580
New best model saved with loss: 1.0580


Epoch 2/100: 100%|██████████| 12/12 [00:08<00:00,  1.35it/s]


Epoch 2/100 Loss: 0.4931
New best model saved with loss: 0.4931


...

Epoch 98/100 Loss: 0.3019


Epoch 99/100: 100%|██████████| 12/12 [00:08<00:00,  1.39it/s]


Epoch 99/100 Loss: 0.3261


Epoch 100/100: 100%|██████████| 12/12 [00:08<00:00,  1.40it/s]


Epoch 100/100 Loss: 0.3309
Checkpoint saved at epoch 100
Final model saved successfully!


In [7]:
from google.colab import files
files.download('/content/best_summit_seeker_model.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>